In [9]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as scipy
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from mne.io import concatenate_raws

In [10]:
import mne
from mne.datasets.sleep_physionet.age import fetch_data

VAMOS USAR 4 pessoas para treinar o  modelo, 2 para teste

**_Direito para tudo_** 

In [14]:
subjects_train =list(range(40)); subjects_train.remove(36)
subjects_test = list(range(40, 82)); subjects_train.remove(39)
sub_train =  fetch_data(subjects = subjects_train, recording=[1])
sub_test =  fetch_data(subjects = subjects_test, recording=[1])

Using default location ~/mne_data for PHYSIONET_SLEEP...


100%|█████████████████████████████████████| 50.5M/50.5M [00:00<00:00, 43.8GB/s]
100%|█████████████████████████████████████| 2.62k/2.62k [00:00<00:00, 2.13MB/s]
100%|█████████████████████████████████████| 51.3M/51.3M [00:00<00:00, 18.4GB/s]
100%|█████████████████████████████████████| 3.63k/3.63k [00:00<00:00, 1.57MB/s]
100%|█████████████████████████████████████| 51.0M/51.0M [00:00<00:00, 33.9GB/s]
100%|█████████████████████████████████████| 4.31k/4.31k [00:00<00:00, 3.35MB/s]
100%|█████████████████████████████████████| 49.8M/49.8M [00:00<00:00, 45.5GB/s]
100%|█████████████████████████████████████| 4.47k/4.47k [00:00<00:00, 3.51MB/s]
 92%|█████████████████████████████████▉   | 45.5M/49.6M [02:19<04:53, 14.0kB/s]

Análise da qualidade dos dados, em raw


In [ ]:
def signal_stage(subjects, sub):
     # guarda todos os sinais originais
    signal_EEG = [None] * len(subjects)
    signal_EOG = [None] * len(subjects) # estes são os sinais que pretendemos, para cada utilizador
    annot = [None] * len(subjects) #anotções dos sinais

    events_id = {
    "Sleep stage W": 1,
    "Sleep stage 1": 2,
    "Sleep stage 2": 3,
    "Sleep stage 3": 4,
    "Sleep stage 4": 5,
    "Sleep stage R": 6}


    for i in range(0, len(subjects)):
        signal =  mne.io.read_raw_edf(
            sub[i][0],
            stim_channel="Event marker",
            infer_types=True,
            preload=True,
            verbose="error",  # ignore issues with stored filter settings
        )
        a = signal.copy()
        b = signal.copy()
        signal_EEG[i] = a.pick(["eeg"]).load_data()
        signal_EOG[i] = b.pick(["eog"]).load_data()

        signal_EEG[i].set_eeg_reference("average", projection=True)
        annot[i] =  mne.read_annotations(sub[i][1])
        #signal_EEG[i].hat(annot[i], emit_warning=False)
        signal_EOG[i].set_annotations(annot[i], emit_warning=False)
   
    return(signal_EEG,signal_EOG , annot)

In [ ]:
[EEG_train, EOG_train , Annot_train] = signal_stage(subjects_train, sub_train)
[EEG_test, EOG_test , Annot_test] = signal_stage(subjects_test, sub_test)


In [ ]:
EEG_train[1].info

In [ ]:
EOG_train[1].info

In [ ]:
def signal_representation (subjects, signal, annot):
    events_id = {
    "Sleep stage W": 1,
    "Sleep stage 1": 2,
    "Sleep stage 2": 3,
    "Sleep stage 3": 4,
    "Sleep stage 4": 5,
    "Sleep stage R": 6}

    event_signal = [None] * len(subjects) 
    for i in range(0, len(subjects)):
        signal[i].plot(
                start=60,
                duration=60,
                scalings=dict(eeg=1e-4, resp=1e3, eog=1e-4, emg=1e-7, misc=1e-1))

        # janelas de 30 segundos e observar localização de  eventos temporalmente
        annot[i].crop(annot[i][1]["onset"] - 30 * 60, annot[i][-2]["onset"] + 30 * 60)
        signal[i].set_annotations(annot[i], emit_warning=False)

        event_signal[i], _ = mne.events_from_annotations(
        signal[i], event_id=events_id, chunk_duration=30.0)
        
        fig = mne.viz.plot_events(
            event_signal[i],
            event_id=events_id,
            sfreq=signal[i].info["sfreq"],
            first_samp= event_signal[i][0, 0], on_missing="ignore")
       
    return(event_signal)

        
        
    

In [ ]:
events_EEG_train = signal_representation(subjects_train,EEG_train, Annot_train)
events_EEG_test = signal_representation(subjects_test,EEG_test, Annot_test)



In [ ]:
events_EOG_train = signal_representation(subjects_train,EOG_train, Annot_train)
events_EOG_test = signal_representation(subjects_test,EOG_test, Annot_test)

In [ ]:
def signal_epochs(subjects,signal,event_signal ):
    
    events_id = {
    "Sleep stage W": 1,
    "Sleep stage 1": 2,
    "Sleep stage 2": 3,
    "Sleep stage 3": 4,
    "Sleep stage 4": 5,
    "Sleep stage R": 6}

    epochs = [None] * len(subjects)

    for i in range(0, len(subjects)):
        tmax = 30.0 - 1.0 / signal[i].info["sfreq"] 

        epochs[i] = mne.Epochs(
            raw=signal[i],
            events=event_signal[i],
            event_id=events_id,
            tmin=0.0,
            tmax=tmax,
            baseline=None, 
            on_missing = "ignore"
        )
    epochs = mne.concatenate_epochs(epochs)
    return(epochs)


In [ ]:
epoch_EEG_train = signal_epochs(subjects_train,EEG_train,events_EEG_train)
epoch_EOG_train = signal_epochs(subjects_train,EOG_train,events_EOG_train)


In [ ]:
epoch_EEG_test= signal_epochs(subjects_test,EEG_test,events_EEG_test)
epoch_EOG_test = signal_epochs(subjects_test,EOG_test,events_EOG_test)

**Exemplo: visualizar EEG com eventos marcado, 3 epochs**

In [ ]:
def eeg_power_band(epochs):
    """EEG relative power band feature extraction.

    This function takes an ``mne.Epochs`` object and creates EEG features based
    on relative power in specific frequency bands that are compatible with
    scikit-learn.

    Parameters
    ----------
    epochs : Epochs
        The data.

    Returns
    -------
    X : numpy array of shape [n_samples, 5 * n_channels]
        Transformed data.
    """
    # specific frequency bands
    FREQ_BANDS = {
        "delta": [0.5, 4.5],
        "theta": [4.5, 8.5],
        "alpha": [8.5, 11.5],
        "sigma": [11.5, 15.5],
        "beta": [15.5, 30],
    }

    spectrum = epochs.compute_psd(picks="eeg", fmin=0.5, fmax=30.0)
    psds, freqs = spectrum.get_data(return_freqs=True)
    # Normalize the PSDs
    psds /= np.sum(psds, axis=-1, keepdims=True)

    X = []
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:, :, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)
        X.append(psds_band.reshape(len(psds), -1))

    return np.concatenate(X, axis=1)

tentando com o random forest Classifier

In [ ]:
pipe1 = make_pipeline(
    FunctionTransformer(eeg_power_band, validate=False),
    RandomForestClassifier(n_estimators=100, random_state=42),
)



In [ ]:
y_EEG_train = epoch_EEG_train.events[:,2]
pipe1.fit(epoch_EEG_train, y_EEG_train)


## eeg train merdas - colocar - em vez de fit, meter a fit transform

In [ ]:
y_pred = pipe1.predict(epoch_EEG_test)
y_test = epoch_EEG_test.events[:, 2]
acc = accuracy_score(y_test, y_pred)
print(acc)

